In [1]:
# CAMELS-TNG bias for Figure 5

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from nbodykit.lab import *
from nbodykit import setup_logging, style
from nbodykit.cosmology import Planck15
from scipy.interpolate import InterpolatedUnivariateSpline as ius
import warnings
import h5py
import hdf5plugin
warnings.simplefilter(action='ignore', category=FutureWarning) # many annoying dask warnings from bigfile
plt.style.use(style.notebook)
setup_logging()
L=25. #Mpc/h
volume=L**3

In [2]:
z0,s80 = 1.0,0.8
# these are determined by what is avaliable...
zstep,s8step = 0.05,0.04
zm,zp,s8m,s8p = z0-zstep,z0+zstep,np.round(s80-s8step,2),np.round(s80+s8step,2)

In [3]:
path = '/pscratch/sd/j/jsull/fnl/camels_tng/'
camels_s8s = dict(zip(["1P_2_{0}/".format("n{0}".format(-i) if i<0 else "{0}".format(i)) for i in range(-5,6)],
                      np.linspace(0.6,1.0,11),
                      ))
s8s_camels = {np.round(v,2): k for k, v in camels_s8s.items()}


In [4]:
dirm,dir0,dirp = s8s_camels[s8m],s8s_camels[s80],s8s_camels[s8p]

In [5]:
# get ns separately 
with open(path+dir0+'ICs/CAMB.params') as icfile: lines = icfile.readlines() 
for line in lines: 
    if('an = [' in line): ns = float(line.split('[')[1].split(',')[0])

In [6]:
f0 = h5py.File(path+dir0+'snap_018.hdf5','r')
fzm = h5py.File(path+dirm+'snap_017.hdf5','r')
fzp = h5py.File(path+dirp+'snap_019.hdf5','r')

fof0 = h5py.File(path+dir0+'fof_subhalo_tab_018.hdf5','r')
fofm = h5py.File(path+dirm+'fof_subhalo_tab_018.hdf5','r')
fofp = h5py.File(path+dirp+'fof_subhalo_tab_018.hdf5','r')

fof0zm = h5py.File(path+dir0+'fof_subhalo_tab_017.hdf5','r')
fof0zp = h5py.File(path+dir0+'fof_subhalo_tab_019.hdf5','r')

In [ ]:
cosmo = cosmology.Cosmology(h=f0['Header'].attrs['HubbleParam'], 
                            Omega0_b=f0['Header'].attrs['OmegaBaryon'], 
                            Omega0_cdm=f0['Header'].attrs['Omega0']-f0['Header'].attrs['OmegaBaryon'], # assume no neutrinos 
                            n_s=ns, sigma8=s80)

Dzs = (cosmo.scale_independent_growth_factor(fzm['Header'].attrs['Redshift']),
       cosmo.scale_independent_growth_factor(f0['Header'].attrs['Redshift']),
       cosmo.scale_independent_growth_factor(fzp['Header'].attrs['Redshift']))
s8s = (s8m,s80,s8p)

Set color and mass bins, and bin the data

In [ ]:
#FIXME this is pretty hacky, should set the mask once rather than remasking
def cut_finite(data):
    # Drop subhalos with no stellar mass, or nan values
    stellar_mass = [data[i]['Subhalo']['SubhaloMassType'][:,4]*1e10 for i in range(len(data))]# stars is PartType4
    mass_flag = [np.sort(np.where(stellar_mass[i]>0.0)[0]) for i in range(len(data))]
    grimag = [np.asarray(data[i]['Subhalo']['SubhaloStellarPhotometrics'][:,4:7])[mass_flag[i]]  for i in range(len(data))]
    halo_index = [np.asarray(data[i]['Subhalo']["SubhaloGrNr"])[mass_flag[i]]  for i in range(len(data))]
    halo_mass200m = [np.asarray(data[i]['Group']["Group_M_Mean200"])[halo_index[i]]*1e10  for i in range(len(data))]
    total_mass = [np.sum(data[i]['Subhalo']['SubhaloMassType'],axis=1)*1e10 for i in range(len(data))]
    stellar_mass = [stellar_mass[i][mass_flag[i]] for i in range(len(data))]
    pos_cm = [np.asarray(data[i]['Subhalo']['SubhaloCM'])[mass_flag[i]]/1e3 for i in range(len(data))]
    pos_pot = [np.asarray(data[i]['Subhalo']['SubhaloPos'])[mass_flag[i]]/1e3 for i in range(len(data))]
    halo_mass_flag = [np.where((halo_mass200m[i]>0.0) & (np.isfinite(halo_mass200m[i])))[0] for i in range(len(data))]
    grimag = [grimag[i][halo_mass_flag[i]]  for i in range(len(data))]
    stellar_mass = [stellar_mass[i][halo_mass_flag[i]] for i in range(len(data))]
    total_mass = [total_mass[i][halo_mass_flag[i]] for i in range(len(data))]
    pos_cm = [pos_cm[i][halo_mass_flag[i]] for i in range(len(data))]
    pos_pot = [pos_pot[i][halo_mass_flag[i]] for i in range(len(data))]
    halo_mass200m = [halo_mass200m[i][halo_mass_flag[i]] for i in range(len(data))]
    gminusr = [grimag[i][:,0]-grimag[i][:,1]  for i in range(len(data))]
    return total_mass,stellar_mass, gminusr, halo_mass200m,pos_cm,pos_pot

In [11]:
data_s8,data_Dz = [fofm,fof0,fofp],[fof0zm,fof0,fof0zp]
total_mass_s8, stellar_mass_s8, gminusr_s8, halo_mass200m_s8,pos_cm_s8,pos_pot_s8 = cut_finite(data_s8)
total_mass_Dz, stellar_mass_Dz, gminusr_Dz, halo_mass200m_Dz,pos_cm_Dz,pos_pot_Dz = cut_finite(data_Dz)

Compute number density in each bin in the FD snapshots (simply by counting galaxies)

In [13]:
def bphi(Ns,Xs):
    #X = sigma8 or Dz
    # Function to compute (centered) FD derivative given some numberdensity tuple (Nm,N0,Np)
    # and corresponding redshift/s8 tuple (zm,z0,zp) in order of decreasing redshift (increasing a)
    # or increasing sigma8
    d1pX1,d1pX2 = Xs[0]-Xs[1],-(Xs[1]-Xs[2])
    reld1pX1,reld1pX2 = d1pX1/(Xs[1]),d1pX2/(Xs[1])
    bphi_hi = 2/reld1pX1 *(Ns[0]/Ns[1] -1) #low on N refers to low g-r, redshifts in dec order
    bphi_lo = 2/reld1pX2 *(Ns[2]/Ns[1] -1) 
    #averaging procedure of Barriera++20
    bphi = (bphi_hi + bphi_lo)/2.0
    return bphi


In [ ]:
def bin_mask(pos,binsi,binsj,binsk):
    bin0i,bin1i = binsi
    bin0j,bin1j = binsj
    bin0k,bin1k = binsk
    return (((pos[:,0]>=bin1i) | (pos[:,0]<bin0i) ) |
            ((pos[:,1]>=bin1j) | (pos[:,1]<bin0j) ) |
            ((pos[:,2]>=bin1k) | (pos[:,2]<bin0k) ) )

def jackknife_bphi(position_m,position_0,position_p,
                   Xs, L,L_jk):
    # cut the box into (L/L_jk)**3 subboxes, return mean and variance of 
    # mean number density after dropping once subbox at a time
    V= L**3
    V_jk = V-(L_jk)**3
    N_sub_1d = int(L//L_jk)
    assert V%L_jk**3==0.0
    N_sub = N_sub_1d**3
    bins = np.linspace(0,L,N_sub_1d+1)
    rep_jk = np.zeros((N_sub_1d,N_sub_1d,N_sub_1d))
    for i in range(N_sub_1d):
        for j in range(N_sub_1d):
            for k in range(N_sub_1d):
                mask_m = bin_mask(position_m,bins[i:i+2],bins[j:j+2],bins[k:k+2])
                mask_0 = bin_mask(position_0,bins[i:i+2],bins[j:j+2],bins[k:k+2])
                mask_p = bin_mask(position_p,bins[i:i+2],bins[j:j+2],bins[k:k+2])
                n_m,n_0,n_p = np.sum(mask_m)/V_jk,np.sum(mask_0)/V_jk,np.sum(mask_p)/V_jk
                rep_jk[i,j,k] = bphi((n_m,n_0,n_p),Xs)
                
    mean =  np.mean(rep_jk)
    var = (N_sub-1)*np.mean((rep_jk-mean)**2)
    return mean,np.sqrt(var)


In [ ]:
def plot_bias(gminusr,
              halo_mass200m,
              Xs,
              mass_bin=[10**11.5,10**12.0],
              gmr_bins = np.linspace(0.1,0.75,8),label=None,
              jk=False,position=None):

    gmr_bincens = (gmr_bins[1:]+gmr_bins[:-1])/2
    Ngmrall = []
    Ngmr_bin = []
    mask_bin = []
    for i in range(3):
        bin_gminusr = []#
        mask_gminusr = []
        Ngmr_bins = np.zeros(len(gmr_bins)-1)
        for j in range(len(gmr_bins)-1):
            mask_gminusr.append(((halo_mass200m[i]>mass_bin[0]) & (halo_mass200m[i]<=mass_bin[1])) &
                                                ((gminusr[i]>gmr_bins[j]) & (gminusr[i]<=gmr_bins[j+1])))
            bin_gminusr_j = gminusr[i][mask_gminusr[j] ]
            bin_gminusr.append(bin_gminusr_j)
            Ngmr_bins[j] = np.sum(bin_gminusr_j)
            
        Ngmr_bin.append(Ngmr_bins)
        mask_bin.append(mask_gminusr)
        Ngmrall.append(np.sum(Ngmr_bins))
    if jk:
        gmr_jk_mu,gmr_jk_err = np.zeros(len(gmr_bins)-1),np.zeros(len(gmr_bins)-1)
        for j in range(len(gmr_bins)-1):
            gmr_jk_mu[j],gmr_jk_err[j] = jackknife_bphi(*[position[i][mask_bin[i][j]] for i in range(3)],
                Xs,L,5.0)

    Ngmr_bin = np.asarray(Ngmr_bin)
    bphis = [bphi(Ngmr_bin[:,j],Xs) for j in range(len(gmr_bincens))]
    if jk:
        plt.errorbar(gmr_bincens,gmr_jk_mu,gmr_jk_err,marker='s',label=label,capsize=3,capthick=2)
    else:
        plt.plot(gmr_bincens,bphis,marker='s',label=label)
    plt.ylabel(r'$b_{\phi}$',fontsize=18)
    plt.xlabel(r'$g-r$',fontsize=18)
    plt.title(r'$\log M_{h} \in $'+' [{0:.1f},{1:.1f}]'.format(*np.log10(mass_bin)))


In [ ]:
massb = [10**12.5,10**12.5]
plot_bias(gminusr_s8,halo_mass200m_s8,s8s,mass_bin=massb,label=r"$\sigma_{8}$",jk=True,position=pos_cm_s8)
plot_bias(gminusr_Dz,halo_mass200m_Dz,Dzs,mass_bin=massb,label=r"$D(z)$",jk=True,position=pos_cm_Dz)
plt.legend()
plt.xlabel(r'$g-r$',fontsize=18)
plt.ylabel(r'$b_{\phi}$',fontsize=18)
